In [6]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import green_tsetlin as gt
import numpy as np

Load the **iris** dataset from sklearn.

In [7]:
iris = load_iris()

x = iris['data']
y = iris['target']

In [22]:
x.shape

(150, 4)

As features of this dataset is continuous, we will need to convert it to TM friendly binary data.

In [26]:
# get max and min for each feature
x_max = np.max(x, axis=0)
x_min = np.min(x, axis=0)

# each feature will have 4 intervals
intervals = (x_max - x_min) / 4

# Create 4 intervals for each feature
intervals_list = [[x_min[i] + k * intervals[i] for k in range(4)] for i in range(x.shape[1])]

# Create 4 bit representation for each feature
x_empty = np.zeros((x.shape[0], x.shape[1] * 4)).astype(np.uint8)

for i in range(x.shape[0]):
    for j in range(x.shape[1]):

        x_empty[i, j*4:(j+1)*4] = np.array([1 if x[i, j] >= intervals_list[j][k] else 0 for k in range(4)])

x = x_empty

[1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0]
[5.1 3.5 1.4 0.2]


In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)